#####################################
##    뉴스 요약 트랜스포머 모델   ##
#####################################

0. 인코딩 타입 지정, 라이브러리 임포트, GPU 확인

In [138]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.callbacks import Callback, ModelCheckpoint
from keras.models import load_model
import pickle

gpus = tf.config.list_physical_devices('GPU')
print("Available GPUs:", gpus)

Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [139]:
import pickle

# sentences.pkl 파일에서 데이터 로드
with open('sentences.pkl', 'rb') as f:
    question_list_tf = pickle.load(f)

# abs.pkl 파일에서 데이터 로드
with open('abs.pkl', 'rb') as f:
    answer_list_tf = pickle.load(f)

print("Pickle 파일 로드 완료")

# 로드한 데이터를 확인 (원하는 경우)
print(f'첫 번째 질문: {question_list_tf[0]}')
print(f'첫 번째 답변: {answer_list_tf[0]}')


Pickle 파일 로드 완료
첫 번째 질문: [16369 12006  1250  2802  1905 16159  3908 16159  3907     5  1988   244
 12006  1250  2802   575  5664    64    86 16370     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0]
첫 번째 답변: [16369   111    33     1 12006  1250  1842  1905 16159  3908 16159  3907
     5  1988   244 12006  1250  2802   575  5664    64   128   356  3723
     9  3308  4557     6  3572   356  3723     6  8294 16145  8012    62
    28  2305   968  1250  2621  2399  2066     1   207  3639  4865 14636
   597   355 15455    16  3344  6692 13802 16159 16370     0     0    

1. 전처리 데이터 로드

In [140]:
import pickle
import numpy as np
import tensorflow as tf

# Load data from pickle files
with open(r'C:\TJ_FInal_Project\Gina\sentences.pkl', 'rb') as f:
    sentences = pickle.load(f)

with open(r'C:\TJ_FInal_Project\Gina\abs.pkl', 'rb') as f:
    abs = pickle.load(f)

# Convert lists to numpy arrays if necessary
sentences = np.array(sentences)  # Ensure sentences is a numpy array
abs = np.array(abs)  # Ensure abs is a numpy array

# Verify the shapes of the arrays
print(f'Sentences shape: {sentences.shape}')
print(f'Abs shape: {abs.shape}')

# Create TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': sentences,
        'dec_inputs': abs[:, :-1]  # Ensure abs is sliced properly
    },
    {
        'outputs': abs[:, 1:]
    }
))

# Print examples from the loaded data
print(f'sentence : {sentences[1]}')
print(f'\nabs : {abs[1]}')

# Function to convert data to int16
def convert_to_int16(inputs, outputs):
    inputs = {key: tf.cast(tf.convert_to_tensor(value), tf.int16) for key, value in inputs.items()}
    outputs = {key: tf.cast(tf.convert_to_tensor(value), tf.int16) for key, value in outputs.items()}
    return inputs, outputs

# Apply the conversion function
dataset = dataset.map(lambda inputs, outputs: convert_to_int16(inputs, outputs))

# Check the dataset
for batch in dataset.take(1):
    print({key: value.dtype for key, value in batch[0].items()})
    print({key: value.dtype for key, value in batch[1].items()})


Sentences shape: (111928, 99)
Abs shape: (111928, 199)
sentence : [16369     4   477  1075   375   477  1075  1630  5705     2   140  3422
  2095  6621 15533 16145  8625   148    86 16370     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0]

abs : [16369   111    33     1     4   477  1075  1010   477  1075  1630  5705
     2   140  3422  2095  6621 15533 16145  8625   148   128  3422   953
  2751   405   342   188  8742     2  4607   761    55 16159   711 14528
 16145 10655   188  6263   485   948   282   769    74  3496 15364    16
 11312    97   211  3818 16145 

2. 하이퍼파라미터 설정

In [141]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.load_from_file('C:/TJ_FInal_Project/Gina/tokenizer')

SEN_MAX_LENGTH = 299
ABS_MAX_LENGTH = 799

VOCAB_SIZE = tokenizer.vocab_size + 2
BATCH_SIZE = 32
BUFFER_SIZE = 20000

D_MODEL = 256
NUM_LAYERS = 2  # 1로 바꿔보기
NUM_HEADS = 4
DFF = 256
DROPOUT = 0.3

# D_MODEL = 256
# NUM_LAYERS = 2
# NUM_HEADS = 8
# DFF = 512

EPOCHS = 2000

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

3. 트랜스포머 모델 빌드

3-1. 포지셔널 인코딩

In [142]:
class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    sines = tf.math.sin(angle_rads[:, 0::2])

    cosines = tf.math.cos(angle_rads[:, 1::2])

    angle_rads = np.zeros(angle_rads.shape)
    angle_rads[:, 0::2] = sines
    angle_rads[:, 1::2] = cosines
    pos_encoding = tf.constant(angle_rads)
    pos_encoding = pos_encoding[tf.newaxis, ...]

    print(pos_encoding.shape)
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

3-2. 패딩 마스크

In [143]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  return mask[:, tf.newaxis, tf.newaxis, :]

3-3. 룩-어헤드 마스크

In [144]:
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)

3-4. 셀프 어텐션(스케일드 닷 프로덕트 어텐션)

In [145]:
def scaled_dot_product_attention(query, key, value, mask):
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  if mask is not None:
    logits += (mask * -1e9)

  attention_weights = tf.nn.softmax(logits, axis=-1)

  output = tf.matmul(attention_weights, value)

  return output, attention_weights

3-5. 멀티헤드 어텐션

In [146]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    outputs = self.dense(concat_attention)

    return outputs

3-6. 인코더 층

In [147]:
def encoder_layer(dff, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs, 'key': inputs, 'value': inputs,
          'mask': padding_mask
      })

  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

3-7. 인코더

In [148]:
def encoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = encoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

3-8. 디코더 층

In [149]:
def decoder_layer(dff, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")

  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")

  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs, 'key': inputs, 'value': inputs,
          'mask': look_ahead_mask
      })

  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1, 'key': enc_outputs, 'value': enc_outputs,
          'mask': padding_mask
      })

  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

3-9. 디코더

In [150]:
def decoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')

  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)
  
tf.keras.backend.clear_session()

3-10. 트랜스포머

In [151]:
def transformer(vocab_size, num_layers, dff,
                d_model, num_heads, dropout,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask, output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)
  enc_outputs = encoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[inputs, enc_padding_mask]) 

  dec_outputs = decoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)
  
  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

3-11. 모델 구조 선언

In [152]:
model = transformer(
  vocab_size=VOCAB_SIZE,
  num_layers=NUM_LAYERS,
  dff=DFF,
  d_model=D_MODEL,
  num_heads=NUM_HEADS,
  dropout=DROPOUT
)

(1, 16371, 256)
(1, 16371, 256)


4. 커스텀 스케줄러

In [153]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=1000):
    super(CustomSchedule, self).__init__()
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)
    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)
  
learning_rate = CustomSchedule(D_MODEL)


5. 오차 함수 및 옵티마이저

In [154]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, ABS_MAX_LENGTH - 1))

  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

optimizer = tf.keras.optimizers.Adam(
  learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9
)

6. 정확도 계산 함수 및 컴파일

In [155]:
def acuuracy(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, ABS_MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[acuuracy])

7. 검증 함수

In [156]:
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size+1]

def evaluate(sentence):
  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  for i in range(ABS_MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)

8. 예측 함수

In [157]:
def predict(sentence):
  prediction = evaluate(sentence)

  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])
  print(f'원문 : {sentence}')
  print(f'\n요약 : {predicted_sentence}')

  return predicted_sentence

9. 검증 데이터 준비 및 epoch 마다 검증

In [158]:
df = pd.read_csv('C:\TJ_FInal_Project\Gina\processed_jisikin_final.csv')
sentence = df['Question']
abs = df['Answer']

predict_sentence_1 = sentence[100]
predict_sentence_2 = sentence[200]
predict_abs_1 = abs[100]
predict_abs_2 = abs[200]

previous_loss = tf.Variable(float('inf'), trainable=False)

class EpochValidation(Callback):
    def on_epoch_end(self, epoch, logs=None):
        self.epochPrint(epoch, logs)

    def epochPrint(self, epoch, logs=None):
        if epoch % 10 == 0:
            print("\n***************  첫번째 예측  ***************")
            predict(predict_sentence_1)
            print(f'\n정답 : {predict_abs_1}')
            print("\n***************  두번째 예측  ***************")
            predict(predict_sentence_2)
            print(f'\n정답 : {predict_abs_2}')
        
        current_loss = logs.get('loss')
        
        if current_loss is not None:
            if current_loss < previous_loss.numpy():
                model.save_weights('./transformer.h5')
                print(f'\n손실 값 감소! 이전 손실: {previous_loss.numpy()}, 현재 손실: {current_loss}')
            else:
                print(f'\n손실 값 증가 또는 동일')
            
            previous_loss.assign(current_loss)

10. 학습 및 모델 저장

In [159]:
model.fit(dataset, epochs=EPOCHS, callbacks=[EpochValidation()]) #checkpoint
#model.load_weights('./trasformer.h5')

Epoch 1/2000


ValueError: in user code:

    File "c:\ProgramData\anaconda3\envs\dl-dev\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\F-3\AppData\Local\Temp\ipykernel_15652\4090593889.py", line 3, in acuuracy  *
        return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)
    File "c:\ProgramData\anaconda3\envs\dl-dev\lib\site-packages\keras\metrics\metrics.py", line 3669, in sparse_categorical_accuracy
        matches = metrics_utils.sparse_categorical_matches(y_true, y_pred)
    File "c:\ProgramData\anaconda3\envs\dl-dev\lib\site-packages\keras\utils\metrics_utils.py", line 970, in sparse_categorical_matches
        matches = tf.cast(tf.equal(y_true, y_pred), backend.floatx())

    ValueError: Dimensions must be equal, but are 98 and 198 for '{{node Equal}} = Equal[T=DT_FLOAT, incompatible_shape_error=true](Reshape, Cast_2)' with input shapes: [?,98], [?,198].
